# WaveResNet vs ResPSANN on Synthetic Data

This notebook constructs a simple decaying sine dataset, then fits `ResPSANNRegressor` and `WaveResNetRegressor` with and without trainable sine parameters (amplitude, frequency, decay). Results highlight how allowing these parameters to learn can impact accuracy.


In [ ]:
%matplotlib inline
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt

from psann.sklearn import ResPSANNRegressor, WaveResNetRegressor

torch.manual_seed(0)
np.random.seed(0)


In [ ]:
def generate_synthetic(
    n_samples=2000,
    noise=0.05,
    seed=0,
    shape='sine',
    shape_kwargs=None,
):
    """Generate 1D regression data following several waveform families.

    Args:
        n_samples: Total samples to draw uniformly from [-2π, 2π].
        noise: Standard deviation of additive Gaussian noise.
        seed: Seed for the NumPy RNG.
        shape: Name of the target pattern ("sine", "sawtooth", "chirp", "polynomial").
        shape_kwargs: Optional dict with extra parameters per shape variant.
    """
    rng = np.random.default_rng(seed)
    x = rng.uniform(-2 * np.pi, 2 * np.pi, size=(n_samples, 1)).astype(np.float32)
    base = x[:, 0]
    shape_kwargs = dict(shape_kwargs or {})

    if shape == 'sine':
        amplitude = 1.0 + 0.5 * np.sin(0.5 * base)
        frequency = 1.5 + 0.3 * np.cos(0.25 * base)
        damping = 0.03 + 0.01 * base**2
        signal = amplitude * np.exp(-damping * np.abs(base)) * np.sin(frequency * base)
    elif shape == 'sawtooth':
        freq = float(shape_kwargs.get('frequency', 0.8))
        slope = float(shape_kwargs.get('slope', 1.0))
        period = (2 * np.pi) / max(freq, 1e-3)
        phase = (base / period) + float(shape_kwargs.get('phase', 0.0))
        ramp = 2.0 * (phase - np.floor(phase + 0.5))
        amplitude = 1.0 + 0.4 * np.sin(0.3 * base)
        signal = slope * amplitude * ramp
    elif shape == 'chirp':
        freq0 = float(shape_kwargs.get('freq0', 0.5))
        freq1 = float(shape_kwargs.get('freq1', 2.0))
        base_min, base_max = float(base.min()), float(base.max())
        norm = (base - base_min) / (base_max - base_min + 1e-6)
        freq = freq0 + (freq1 - freq0) * norm
        signal = np.sin(freq * base**2)
    elif shape == 'polynomial':
        coeffs = shape_kwargs.get('coeffs', (0.6, -0.3, 0.05))
        signal = np.zeros_like(base)
        for power, coeff in enumerate(coeffs, start=1):
            signal += float(coeff) * base**power
        signal = np.tanh(signal)
    else:
        raise ValueError(
            f"Unsupported shape '{shape}'. Choose from 'sine', 'sawtooth', 'chirp', or 'polynomial'."
        )

    noise_term = rng.normal(scale=noise, size=signal.shape).astype(np.float32)
    y = (signal + noise_term).astype(np.float32)
    return x, y.reshape(-1, 1)

DATA_SHAPE = 'sine'  # Try: 'sawtooth', 'chirp', 'polynomial'
SHAPE_KWARGS = {}

X, y = generate_synthetic(
    n_samples=3000,
    noise=0.05,
    seed=42,
    shape=DATA_SHAPE,
    shape_kwargs=SHAPE_KWARGS,
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape} (pattern: {DATA_SHAPE})")

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(X_train[:, 0], y_train[:, 0], s=8, alpha=0.35, label='train')
ax.scatter(X_test[:, 0], y_test[:, 0], s=12, alpha=0.6, label='test')
ax.set_title(f"Synthetic target pattern: {DATA_SHAPE}")
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend(loc='upper right')
plt.show()


In [ ]:
def evaluate_psann(label, factory):
    model = factory()
    start = time.time()
    model.fit(X_train, y_train)
    duration = time.time() - start
    preds = np.asarray(model.predict(X_test), dtype=np.float32).reshape(-1, 1)
    y_true = y_test.reshape(-1)
    y_pred = preds.reshape(-1)
    metrics = {
        'model': label,
        'mse': float(mean_squared_error(y_true, y_pred)),
        'r2': float(r2_score(y_true, y_pred)),
        'time_sec': duration,
    }

    def predictor(x):
        arr = np.asarray(x, dtype=np.float32)
        out = np.asarray(model.predict(arr), dtype=np.float32).reshape(-1, 1)
        return out

    return metrics, predictor, model


def evaluate_mlp(label, *, hidden_layers=(128,) * 4, max_iter=400):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    mlp = MLPRegressor(
        hidden_layer_sizes=hidden_layers,
        activation='tanh',
        solver='adam',
        learning_rate_init=5e-4,
        batch_size=256,
        max_iter=max_iter,
        early_stopping=True,
        n_iter_no_change=20,
        random_state=0,
    )

    start = time.time()
    mlp.fit(X_train_scaled, y_train[:, 0])
    duration = time.time() - start

    preds = mlp.predict(X_test_scaled).reshape(-1, 1).astype(np.float32)
    y_true = y_test.reshape(-1)
    y_pred = preds.reshape(-1)

    metrics = {
        'model': label,
        'mse': float(mean_squared_error(y_true, y_pred)),
        'r2': float(r2_score(y_true, y_pred)),
        'time_sec': duration,
    }

    def predictor(x):
        arr = scaler.transform(np.asarray(x, dtype=np.float32))
        out = mlp.predict(arr).reshape(-1, 1).astype(np.float32)
        return out

    return metrics, predictor, {'scaler': scaler, 'model': mlp}


class SimpleLSTMRegressor(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=128, num_layers=2, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
        )
        self.head = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.head(out[:, -1, :])


def evaluate_lstm(
    label,
    *,
    hidden_dim=128,
    num_layers=2,
    epochs=60,
    lr=5e-4,
    batch_size=256,
    dropout=0.1,
):
    model = SimpleLSTMRegressor(
        input_dim=1,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout=dropout,
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    def _to_dataset(X, y):
        features = torch.from_numpy(np.asarray(X, dtype=np.float32)).view(-1, 1, 1)
        targets = torch.from_numpy(np.asarray(y, dtype=np.float32)).view(-1, 1)
        return TensorDataset(features, targets)

    train_loader = DataLoader(_to_dataset(X_train, y_train), batch_size=batch_size, shuffle=True)

    start = time.time()
    for _ in range(epochs):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()
    duration = time.time() - start

    model.eval()
    with torch.no_grad():
        test_tensor = torch.from_numpy(np.asarray(X_test, dtype=np.float32)).view(-1, 1, 1)
        preds = model(test_tensor).detach().numpy().astype(np.float32)

    y_true = y_test.reshape(-1)
    y_pred = preds.reshape(-1)
    metrics = {
        'model': label,
        'mse': float(mean_squared_error(y_true, y_pred)),
        'r2': float(r2_score(y_true, y_pred)),
        'time_sec': duration,
    }

    def predictor(x):
        arr = torch.from_numpy(np.asarray(x, dtype=np.float32)).view(-1, 1, 1)
        model.eval()
        with torch.no_grad():
            out = model(arr).detach().numpy().astype(np.float32)
        return out

    return metrics, predictor, model


activation_trainable = {'learnable': ('amplitude', 'frequency', 'decay')}
activation_frozen = {'learnable': ()}

respsann_common = dict(
    hidden_layers=6,
    hidden_units=128,
    epochs=60,
    batch_size=256,
    lr=5e-4,
    optimizer='adam',
    device='cpu',
    random_state=0,
)

wave_common = dict(
    hidden_layers=6,
    hidden_units=128,
    epochs=60,
    batch_size=256,
    lr=5e-4,
    optimizer='adam',
    device='cpu',
    random_state=0,
)

psann_experiments = [
    ('ResPSANN (trainable)', lambda: ResPSANNRegressor(activation=activation_trainable, activation_type='psann', **respsann_common)),
    ('ResPSANN (frozen)', lambda: ResPSANNRegressor(activation=activation_frozen, activation_type='psann', **respsann_common)),
    ('WaveResNet (trainable)', lambda: WaveResNetRegressor(activation=activation_trainable, activation_type='psann', **wave_common)),
    ('WaveResNet (frozen)', lambda: WaveResNetRegressor(activation=activation_frozen, activation_type='psann', **wave_common)),
]

results = []
trained_models = []
raw_models = {}

for label, factory in psann_experiments:
    metrics, predictor, model = evaluate_psann(label, factory)
    results.append(metrics)
    trained_models.append((label, predictor))
    raw_models[label] = model

mlp_metrics, mlp_predictor, mlp_components = evaluate_mlp('MLP (tanh, 4x128)')
results.append(mlp_metrics)
trained_models.append((mlp_metrics['model'], mlp_predictor))
raw_models[mlp_metrics['model']] = mlp_components

lstm_metrics, lstm_predictor, lstm_model = evaluate_lstm('LSTM (2x128)')
results.append(lstm_metrics)
trained_models.append((lstm_metrics['model'], lstm_predictor))
raw_models[lstm_metrics['model']] = lstm_model

results_df = pd.DataFrame(results).sort_values('mse').reset_index(drop=True)
display(results_df)


In [ ]:
grid = np.linspace(X_train[:, 0].min(), X_train[:, 0].max(), 600, dtype=np.float32).reshape(-1, 1)
fig, ax = plt.subplots(figsize=(9, 4))
ax.scatter(X_test[:, 0], y_test[:, 0], s=12, alpha=0.4, label='test')
for label, predictor in trained_models:
    preds = predictor(grid)
    ax.plot(grid[:, 0], preds[:, 0], label=label)
ax.set_title('Model fits across settings')
ax.set_xlabel('x')
ax.set_ylabel('prediction')
ax.legend()
plt.show()
